In [1]:
import pandas as pd

In [17]:
df = pd.read_csv('jeopardy-topic-modeling/JEOPARDY_CSV.csv')
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [18]:
df[' Round'].unique()

array(['Jeopardy!', 'Double Jeopardy!', 'Final Jeopardy!', 'Tiebreaker'],
      dtype=object)

In [19]:
len(df[df[' Round'] == 'Final Jeopardy!'])

3631

### Clean up data

In [20]:
df.columns = ['show_id', 'date', 'round', 'category', 'value', 'question', 'answer']

In [21]:
#df.value.unique()

In [22]:
#df[df.value == 'None']

In [23]:
df['value'].replace({'None': '$0'}, inplace=True)

In [24]:
df['value'] = df['value'].apply(lambda x: int(x[1:].replace(',',''))) #remove $ remove comma and convert to int

In [26]:
df['year'] = df['date'].apply(lambda x: int(x[:4]))

In [27]:
df.head()

,show_id,date,round,category,value,question,answer,year
0,4680,2004-12-31,Jeopardy!,HISTORY,200,"For the last 8 years of his life, Galileo was ...",Copernicus,2004
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,2004
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200,The city of Yuma in this state has a record av...,Arizona,2004
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,2004
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,2004


In [29]:
df['round'].unique()

array(['Jeopardy!', 'Double Jeopardy!', 'Final Jeopardy!', 'Tiebreaker'],
      dtype=object)

### Explore categories

In [41]:
df.category.value_counts()[:25]

BEFORE & AFTER             547
SCIENCE                    519
LITERATURE                 496
AMERICAN HISTORY           418
POTPOURRI                  401
WORLD HISTORY              377
WORD ORIGINS               371
COLLEGES & UNIVERSITIES    351
HISTORY                    349
SPORTS                     342
U.S. CITIES                339
WORLD GEOGRAPHY            338
BODIES OF WATER            327
ANIMALS                    324
STATE CAPITALS             314
BUSINESS & INDUSTRY        311
ISLANDS                    301
WORLD CAPITALS             300
U.S. GEOGRAPHY             299
RELIGION                   297
OPERA                      294
SHAKESPEARE                294
LANGUAGES                  284
BALLET                     282
TELEVISION                 281
Name: category, dtype: int64

## Attempt at running LDA

In [50]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

In [140]:
import re
import spacy

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [126]:
df.head()

,show_id,date,round,category,value,question,answer,year
0,4680,2004-12-31,Jeopardy!,HISTORY,200,"For the last 8 years of his life, Galileo was ...",Copernicus,2004
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,2004
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200,The city of Yuma in this state has a record av...,Arizona,2004
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,2004
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,2004


In [128]:
# Convert to list
data = df.question.values.tolist()
# Remove Emails
#data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
print(data[:1])

NameError: name 'pprint' is not defined

In [131]:
# Tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['for', 'the', 'last', 'years', 'of', 'his', 'life', 'galileo', 'was', 'under', 'house', 'arrest', 'for', 'espousing', 'this', 'mans', 'theory']]


In [132]:
# Stemming
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [137]:
# Initialize spacy ‘en_core_web_sm’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

['year life galileo be house arrest espouse man theory', 'football star carlisle school mlb season red giant brave']


In [138]:
# Vectorize -- Document-Word matrix
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}') 
# num chars > 3
                             # max_features=50000,             
# max number of uniq words    )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [141]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [145]:
# Create Document — Topic Matrix

#lda_output = lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic


# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)
def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.560000,0.010000,0.010000,0.010000,0.120000,0.010000,0.010000,0.010000,0.010000,0.120000,0.120000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0
Doc1,0.800000,0.010000,0.010000,0.010000,0.110000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0
Doc2,0.010000,0.010000,0.010000,0.450000,0.120000,0.010000,0.010000,0.120000,0.010000,0.120000,0.010000,0.010000,0.010000,0.010000,0.010000,0.120000,0.010000,0.010000,0.010000,0.010000,3
Doc3,0.150000,0.010000,0.010000,0.010000,0.580000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.150000,0.010000,0.010000,0.010000,0.010000,0.010000,4
Doc4,0.010000,0.010000,0.720000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.150000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,2
Doc5,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.880000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,8
Doc6,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.880000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,13
Doc7,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.840000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,8
Doc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.820000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7
Doc9,0.010000,0.010000,0.840000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,2


In [148]:
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.56,0.01,0.01,0.01,0.12,0.01,0.01,0.01,0.01,0.12,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0
Doc1,0.80,0.01,0.01,0.01,0.11,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0
Doc2,0.01,0.01,0.01,0.45,0.12,0.01,0.01,0.12,0.01,0.12,...,0.01,0.01,0.01,0.01,0.12,0.01,0.01,0.01,0.01,3
Doc3,0.15,0.01,0.01,0.01,0.58,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.15,0.01,0.01,0.01,0.01,0.01,4
Doc4,0.01,0.01,0.72,0.01,0.01,0.01,0.01,0.01,0.01,0.15,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,2


In [149]:
len(df_document_topic), len(df)

(216930, 216930)

In [153]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,aaron,aba,abandon,abbey,abbot,abbott,abbreviate,abbreviation,abc,abdicate,...,zip,zodiac,zombie,zone,zoo,zoom,zorba,zorro,zulu,zurich
Topic0,0.05,0.05,0.050000,0.05,0.050000,0.05,0.05,0.05,0.050000,0.050000,...,0.05,0.05,0.05,0.05,0.050000,0.05,0.050000,0.05,0.05,0.050000
Topic1,0.05,0.05,0.050000,0.05,0.050000,0.05,0.05,0.05,0.050000,0.050000,...,0.05,0.05,0.05,0.05,172.324973,0.05,0.050000,0.05,0.05,12.174816
Topic2,0.05,0.05,0.050000,0.05,21.668819,0.05,0.05,0.05,0.050000,50.784336,...,0.05,0.05,0.05,0.05,0.050000,0.05,0.050000,0.05,0.05,0.050000
Topic3,0.05,0.05,64.443097,0.05,0.050000,0.05,0.05,0.05,0.050000,0.050000,...,0.05,0.05,0.05,0.05,0.050000,0.05,0.050000,0.05,0.05,0.050000
Topic4,0.05,0.05,0.050000,0.05,0.050000,0.05,0.05,0.05,87.341105,0.050000,...,0.05,0.05,0.05,0.05,0.050000,0.05,12.367085,0.05,0.05,0.050000


In [155]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,write,include,day,island,letter,life,set,john,family,york,school,wear,law,center,art
Topic 1,know,bear,home,turn,nation,land,leader,england,france,prize,poet,award,fly,artist,flag
Topic 2,come,president,people,tell,die,son,way,dog,singer,boy,child,lady,tale,need,henry
Topic 3,hit,author,feature,hear,death,opera,act,think,party,air,ask,lie,guy,instrument,send
Topic 4,play,year,star,song,line,live,serve,sport,night,language,role,version,host,actress,let
Topic 5,say,novel,end,place,leave,university,girl,tree,college,jame,hall,eye,june,flower,service
Topic 6,water,create,look,plant,building,return,governor,cut,london,develop,peter,kind,port,drama,ring
Topic 7,make,character,animal,lake,sound,jimmy,record,lose,fall,eat,site,date,baby,region,ride
Topic 8,word,title,king,woman,game,bird,sell,share,appear,read,kid,brand,bring,product,metal
Topic 9,state,country,type,time,house,refer,cover,queen,dance,fish,mark,change,grow,shoot,jack


In [151]:
df['topic'] = df_document_topic['dominant_topic'].values
df

,show_id,date,round,category,value,question,answer,year,topic
0,4680,2004-12-31,Jeopardy!,HISTORY,200,"For the last 8 years of his life, Galileo was ...",Copernicus,2004,0
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,2004,0
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,200,The city of Yuma in this state has a record av...,Arizona,2004,3
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,2004,4
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,2004,2
...,...,...,...,...,...,...,...,...,...
216925,4999,2006-05-11,Double Jeopardy!,RIDDLE ME THIS,2000,This Puccini opera turns on the solution to 3 ...,Turandot,2006,3
216926,4999,2006-05-11,Double Jeopardy!,"""T"" BIRDS",2000,In North America this term is properly applied...,a titmouse,2006,0
216927,4999,2006-05-11,Double Jeopardy!,AUTHORS IN THEIR YOUTH,2000,"In Penny Lane, where this ""Hellraiser"" grew up...",Clive Barker,2006,1
216928,4999,2006-05-11,Double Jeopardy!,QUOTATIONS,2000,"From Ft. Sill, Okla. he made the plea, Arizona...",Geronimo,2006,1
